In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from ln_utils import *

In [ ]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

# Load data

In [ ]:
graph_files = []

In [ ]:
data_dir = "../LNdata/lncaptures/lngraph/2019/"
graph_files +=  [data_dir + f for f in sorted(os.listdir(data_dir)) if ".json" in f]
MIN_TIME = 1549065601-86400 #Saturday, February 2, 2019 12:00:01 AM
#MAX_TIME = 1552867201 #Monday, March 18, 2019 12:00:01 AM

In [ ]:
data_dir = "../LNdata/"
#graph_files = [data_dir + f for f in sorted(os.listdir(data_dir)) if ".json" in f]
graph_files += [data_dir + f for f in sorted(os.listdir(data_dir)) if ".json" in f][5:]
#MIN_TIME = 1552478399 # Wednesday, March 13, 2019 11:59:59 AM
MAX_TIME = 1553947199 # Saturday, March 30, 2019 11:59:59 AM

graph_files = graph_files[:15]
#graph_files

# Calculate ranks of connected nodes

In [ ]:
attachments, node_list, scores_list, ranks_list = observe_node_attachements_over_time(graph_files[:-1])

# TODO: export calculated centralities etc. + popularities!!!

# Analyse ranks of connected nodes

In [ ]:
attachments[0].head()

In [ ]:
att = attachments[0]

In [ ]:
def bin_rank(rank):
    if rank < 10:
        return 1
    elif rank < 100:
        return 2
    else:
        return 3

In [ ]:
att["betw_rank_bin"] = att["betw_rank"].apply(bin_rank)

In [ ]:
att["betw_rank"].hist(bins=50)

In [ ]:
att["betw_rank_bin"].hist(bins=50)

In [ ]:
for att in attachments:
    print(att.mean())

### Weekly correlations of popular authority nodes

In [ ]:
pop_df = get_attachement_popularity(attachments)

In [ ]:
sns.heatmap(corr_mx(pop_df, "spearman"), annot=True)

In [ ]:
sns.heatmap(corr_mx(pop_df, "wkendall"), annot=True)

In [ ]:
sns.heatmap(corr_mx(pop_df, "kendall"), annot=True)

In [ ]:
sp = pop_corr_with_centralities(pop_df, scores_list, method="spearman")
ke = pop_corr_with_centralities(pop_df, scores_list, method="kendall")
wk = pop_corr_with_centralities(pop_df, scores_list, method="wkendall")

In [ ]:
plt.figure(figsize=(15,7))
plt.title("Correlation of most popular nodes to node centralities")
x = range(pop_df.shape[1])
plt.plot(x, sp["betw"],label="betw_spearman")
plt.plot(x, sp["wdeg"],label="wdeg_spearman")
plt.plot(x, wk["betw"],"--",label="betw_weighted_kendall")
plt.plot(x, wk["wdeg"],"--",label="wdeg_weighted_kendall")
plt.plot(x, ke["betw"],"-.",label="betw_kendall")
plt.plot(x, ke["wdeg"],"-.",label="wdeg_kendall")
plt.legend()
plt.show()

### Most popular authority nodes:

In [ ]:
most_pop = pop_df.rank(ascending=False).mean(axis=1).sort_values()[:20]

In [ ]:
pop_df.head()

In [ ]:
most_pop

#### Betweeness rank of most popular nodes

In [ ]:
for node in most_pop.index:
    print(ranks_list[-1]["betw"][node])